In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

In [2]:
database ="nucleotide"
word = "ddx18 and homo sapiens and Chromosome 2 and not predicted and not unverified"
res= "30"
Entrez.email="guilherme.lobo@ua.pt"
handle=Entrez.esearch(db = database, term=word, retmax= res)
record=Entrez.read(handle)
handle.close()
idlist= record['IdList']

handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for x in records:
    print(x.id,x.description)

NM_006773.4 Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
AC009312.4 Homo sapiens BAC clone RP11-425F6 from 2, complete sequence
AC009404.5 Homo sapiens BAC clone RP11-28H22 from 2, complete sequence
CM000253.1 Homo sapiens chromosome 2, whole genome shotgun sequence
CH471103.1 Homo sapiens 211000035839809 genomic scaffold, whole genome shotgun sequence
DQ655976.2 Homo sapiens clone Affy08254D01, mRNA sequence
DQ655975.2 Homo sapiens clone Affy08248B12, mRNA sequence
NT_086328.2 Homo sapiens chromosome 2 sequence, ENCODE region ENr121


In [3]:
for rec in records:
    #print(rec)
    if rec.id == "NM_006773.4":
        print("Record sequence length: ", len(rec.seq))
        print("Record features count: ", len(rec.features))
        print( "Nuclleotide sequence:", str(rec.seq)) #useful for blastn
        
        cds = []
        for i,feature in enumerate(rec.features):
            if feature.type=="CDS":
                cds.append(i)
                for k in cds:
                    print("ddx18 gene synonyms:",rec.features[k].qualifiers["gene_synonym"])#can help latter in the search
                    print("name:",rec.features[k].qualifiers["product"])
                    protein_ID=rec.features[k].qualifiers["protein_id"]
                    sequence_protein=rec.features[k].qualifiers["translation"]#get the protein seq of my cds
                    print("Protein sequence:",sequence_protein)
        gene = []
        for j in range(len(rec.features)):
            if rec.features[j].type == "gene":
                gene.append(j)
                for g in gene:
                    print("Location:",rec.features[g].location)
                    print("Strand:",rec.features[g].location.strand)
                    print("Gene:",rec.features[g].qualifiers["gene"])

Record sequence length:  3753
Record features count:  27
Nuclleotide sequence: ACGTGCGGCCGGAAGGGAAGTAACGTCAGCCTGAGAACTGAGTAGCTGTACTGTGTGGCGCCTTATTCTAGGCACTTGTTGGGCAGAATGTCACACCTGCCGATGAAACTCCTGCGTAAGAAGATCGAGAAGCGGAACCTCAAATTGCGGCAGCGGAACCTAAAGTTTCAGGGGGCCTCAAATCTGACCCTATCGGAAACTCAAAATGGAGATGTATCTGAAGAAACAATGGGAAGTAGAAAGGTTAAAAAATCAAAACAAAAGCCCATGAATGTGGGCTTATCAGAAACTCAAAATGGAGGCATGTCTCAAGAAGCAGTGGGAAATATAAAAGTTACAAAGTCTCCCCAGAAATCCACTGTATTAACCAATGGAGAAGCAGCAATGCAGTCTTCCAATTCAGAATCAAAAAAGAAAAAGAAGAAAAAGAGAAAAATGGTGAATGATGCTGAGCCTGATACGAAAAAAGCAAAAACTGAAAACAAAGGGAAATCTGAAGAAGAAAGTGCCGAGACTACTAAAGAAACAGAAAATAATGTGGAGAAGCCAGATAATGATGAAGATGAGAGTGAGGTGCCCAGTCTGCCCCTGGGACTGACAGGAGCTTTTGAGGATACTTCGTTTGCTTCTCTATGTAATCTTGTCAATGAAAACACTCTGAAGGCAATAAAAGAAATGGGTTTTACAAACATGACTGAAATTCAGCATAAAAGTATCAGACCACTTCTGGAAGGCAGGGATCTTCTAGCAGCTGCAAAAACAGGCAGTGGTAAAACCCTGGCTTTTCTCATCCCTGCAGTTGAACTCATTGTTAAGTTAAGGTTCATGCCCAGGAATGGAACAGGAGTCCTTATTCTCTCACCTACTAGAGAACTAGCCATGCAAACCTTTGGTGTTCTTAAGGAGCTGATGACTCACCAC

In [5]:
record = SeqIO.read(open("Gui.fasta"), format="fasta") 
print (len(record.seq))

3755


In [6]:
Blast = NCBIWWW.qblast("blastn", "nt", record.seq)
with open('Gui_bla.xml', "w") as out_handle:
        out_handle.write(Blast.read())
Blast.close()

In [ ]:
with open('fasta_DNA_.xml', "w") as out_handle:
        out_handle.write(sequence_nucleotide_DDX18.read())
sequence_nucleotide_DDX18.close()

In [7]:
results_Blastn = open("Gui_bla.xml")
blastn_records = NCBIXML.read(results_Blastn)
for parameter in blastn_records.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  NM_006773
Definition:  Homo sapiens DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  0.0220792
E-value:  0.0220792

Accession:  BC024739
Definition:  Homo sapiens DEAD (Asp-Glu-Ala-Asp) box polypeptide 18, mRNA (cDNA clone MGC:29902 IMAGE:4995871), complete cds
E-value:  0.0
E-value:  0.0220792
E-value:  0.0220792

Accession:  XM_003819127
Definition:  PREDICTED: Pan paniscus DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  0.000519252
E-value:  0.0220792

Accession:  XM_515753
Definition:  PREDICTED: Pan troglodytes DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  0.0220792
E-value:  0.0220792

Accession:  XM_004031662
Definition:  PREDICTED: Gorilla gorilla gorilla DEAD-box helicase 18 (DDX18), mRNA
E-value:  0.0
E-value:  1.00239e-06
E-value:  0.000519252


In [10]:
Trans=['MSHLPMKLLRKKIEKRNLKLRQRNLKFQGASNLTLSETQNGDVSEETMGSRKVKKSKQKPMNVGLSETQNGGMSQEAVGNIKVTKSPQKSTVLTNGEAAMQSSNSESKKKKKKKRKMVNDAEPDTKKAKTENKGKSEEESAETTKETENNVEKPDNDEDESEVPSLPLGLTGAFEDTSFASLCNLVNENTLKAIKEMGFTNMTEIQHKSIRPLLEGRDLLAAAKTGSGKTLAFLIPAVELIVKLRFMPRNGTGVLILSPTRELAMQTFGVLKELMTHHVHTYGLIMGGSNRSAEAQKLGNGINIIVATPGRLLDHMQNTPGFMYKNLQCLVIDEADRILDVGFEEELKQIIKLLPTRRQTMLFSATQTRKVEDLARISLKKEPLYVGVDDDKANATVDGLEQGYVVCPSEKRFLLLFTFLKKNRKKKLMVFFSSCMSVKYHYELLNYIDLPVLAIHGKQKQNKRTTTFFQFCNADSGTLLCTDVAARGLDIPEVDWIVQYDPPDDPKEYIHRVGRTARGLNGRGHALLILRPEELGFLRYLKQSKVPLSEFDFSWSKISDIQSQLEKLIEKNYFLHKSAQEAYKSYIRAYDSHSLKQIFNVNNLNLPQVALSFGFKVPPFVDLNVNSNEGKQKKRGGGGGFGYQKTKKVEKSKIFKHISKKSSDSRQFSH']

In [11]:
result_handle = NCBIWWW.qblast("blastp", "swissprot", Trans)

In [12]:
with open('GUI_BLAP.xml', "w") as out_handle:
        out_handle.write(result_handle.read())
result_handle.close()

In [14]:
results_Blastp = open("GUI_BLAP.xml")
blastp_records = NCBIXML.read(results_Blastp)
for parameter in blastp_records.alignments[0:5]:
    print()
    print('Accession: ', parameter.accession)
    print('Definition: ', parameter.hit_def)    
    for e_v in parameter.hsps:
        print('E-value: ', e_v.expect)


Accession:  Q9NVP1
Definition:  RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18; AltName: Full=Myc-regulated DEAD box protein; Short=MrDb [Homo sapiens]
E-value:  0.0

Accession:  Q8K363
Definition:  RecName: Full=ATP-dependent RNA helicase DDX18; AltName: Full=DEAD box protein 18 [Mus musculus]
E-value:  0.0

Accession:  Q9VD51
Definition:  RecName: Full=Probable ATP-dependent RNA helicase pitchoune [Drosophila melanogaster]
E-value:  0.0

Accession:  Q1EA54
Definition:  RecName: Full=ATP-dependent RNA helicase HAS1 [Coccidioides immitis RS]
E-value:  0.0

Accession:  A4R8B5
Definition:  RecName: Full=ATP-dependent RNA helicase HAS1 [Pyricularia oryzae 70-15]
E-value:  0.0


In [17]:
results_Blast= open('GUI_BLAP.xml')
blast_records = NCBIXML.read(results_Blast)
for alignment in  blast_records.alignments:
    #print (alignment.hit_id)
    m = alignment.hit_id.replace("sp|","")
    a=m[0:6]
    print (a)
    

Q9NVP1
Q8K363
Q9VD51
Q1EA54
A4R8B5
A1CIQ5
Q5BBY1
Q4IEK8
A1CW14
Q4WQM4
A2Q9T6
Q0UR48
Q54S03
A3LNR6
Q2UUN6
Q7S2N9
Q09916
Q03532
Q6CXB7
Q6BH93
Q6FIL3
Q74Z73
P0CQ84
A5E2Z9
A5DID7
Q6C7D2
Q4P6N0
Q0CZS8
Q5AK59
Q2GMX1
Q84T03
Q9LIH9
Q9SB89
Q0DBS1
Q13206
Q80Y44
Q5ZJF6
A1CTZ2
Q4P5U4
Q1EB31
Q0CMM5
Q4WM60
A1DNF9
A7ESL8
Q2H2J1
Q5BFU1
Q0UMB9
Q2UHB7
A2QS00
Q54Q94


In [ ]:
Q9NVP1
Q8K363
Q9VD51
Q1EA54
A4R8B5
A1CIQ5
Q5BBY1
Q4IEK8
A1CW14
Q4WQM4
A2Q9T6
Q0UR48
Q54S03
A3LNR6
Q2UUN6
Q7S2N9
Q09916
Q03532
Q6CXB7
Q6BH93
Q6FIL3
Q74Z73
P0CQ84

In [ ]:
#saving the CDS_nucleotides and CD_aminoacid seqs in files
filename_CDS_nucl = "CDS_nucleot_GLI2_seq.fasta"
filename_CDS_prot = "CDS_prot_GLI2_seq.fasta"
input_handle  = open(filename, "r")
output_handle_nucl = open(filename_CDS_nucl, "w")
output_handle_prot = open(filename_CDS_prot, "w")
for seq_record in SeqIO.parse(input_handle, "genbank") :
    output_handle_nucl.write(">\n%s" % (CDS_nuc_seq))
    output_handle_prot.write(">\n%s" % ("".join(record.features[position].qualifiers['translation']))) #without join, output is a list
    
output_handle_nucl.close()
output_handle_prot.close()
input_handle.close()

In [ ]:
filename_CDS_nucl = "CDS_nucleot_GLI2_seq.fasta"
output_handle_nucl = open(filename_CDS_nucl, "w")
for seq_record in SeqIO.read(open('NM_006773.4.gb'), "genbank")
    output_handle_nucl.write(seq_record.seq)
output_handle_nucl.close()
input_handle.close()

In [ ]:
record = SeqIO.read(open("NM_006773.4.gb"), format="genbank")
for x in records:
    a= x.seq
with open('fasta_DNA_.xml', "w") as out_handle:
        out_handle.write(record.read())
record.close()